In [ ]:
%pip install gurobipy>=10
import gurobipy as grb
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats

## Q1

In [ ]:
model = grb.Model("B&Q")

VARIABLES = 5
x=model.addVars([i for i in range(VARIABLES)],vtype=grb.GRB.CONTINUOUS,name="x")

### The Constraints are:
1. 長leg和短leg的長度須小於6000(因為這是該公司最大可使用的長度) (Balance Constraint)

2. 桌面的數量需大於2種桌子的產量（這樣才能組裝出這麼多個桌子）

3. 短leg的數量至少要有4倍的桌子A數量（因一張桌子需要4隻腳）

4. 長leg的數量至少要有4倍的桌子Ｂ數量（因一張桌子需要4隻腳）

5. 製作的時間＊該任務的數量最多只能花labor 80小時（根據題目）（Balance Constraint）

In [ ]:
model.addConstr((18*x[2]+30*x[3] <= 6000))
model.addConstr(x[4] >= x[0]+x[1])
model.addConstr(x[2]>=4*x[0])
model.addConstr(x[3]>=4*x[1])
model.addConstr(0.3*x[0]+0.15*x[1]+0.1*x[2]+0.15*x[3]+0.5*x[4] <= 80)
for i in range(5):
  model.addConstr(x[i] >=0)

目標：to maximize its profit（價格＊銷售數量）

In [ ]:
obj = 30*x[0]+45*x[1]
model.setObjective(obj ,grb.GRB.MAXIMIZE)

In [ ]:
model.update()
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 5 columns and 19 nonzeros
Model fingerprint: 0xd90bbe7f
Coefficient statistics:
  Matrix range     [1e-01, 3e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 6e+03]
Presolve removed 8 rows and 3 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+03   1.063960e+02   0.000000e+00      0s
       2    2.3666667e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.366666667e+03


In [ ]:
print(model.Status == grb.GRB.OPTIMAL)
print(model.display())

True
Maximize
  30.0 x[0] + 45.0 x[1]
Subject To
  R0: 18.0 x[2] + 30.0 x[3] <= 6000
  R1: -1.0 x[0] + -1.0 x[1] + x[4] >= 0
  R2: -4.0 x[0] + x[2] >= 0
  R3: -4.0 x[1] + x[3] >= 0
  R4: 0.3 x[0] + 0.15 x[1] + 0.1 x[2] + 0.15 x[3] + 0.5 x[4] <= 80
  R5: x[0] >= 0
  R6: x[1] >= 0
  R7: x[2] >= 0
  R8: x[3] >= 0
  R9: x[4] >= 0
None


ANS1: (因python變數從0開始，所以程式呈現與題目會差一個index，但我還是以題目敘述來回答)
1. x1(for the number of assembled model A): 應生產38.8888888...個

2. x2(for the number of assembled model B): 應生產26.6666666...個

3. x3(for the number of short legs produced): 應生產155.55555...個

4. x4(for the number of long legs produced): 應生產106.66666...個

5. x5(for the number of tabletops produced): 應生產應生產65.55555...個

In [ ]:
for v in model.getVars():
  print(v.VarName, v.X)

x[0] 38.88888888888887
x[1] 26.666666666666675
x[2] 155.5555555555555
x[3] 106.66666666666671
x[4] 65.55555555555554


## Q2

### 7個變數來表示7位player


In [ ]:
m2 = grb.Model("CoachNight")

PLAYER = 7

x = m2.addVars([i for i in range(PLAYER)], vtype = grb.GRB.INTEGER, name = "x")

### Constraints are:
1. 必須要有5位上場

2. 要在能夠打guard的位置裡面選至少4個選手

3. 要在能夠打foward的位置裡面選至少2個選手

4. 要在能夠打center的位置裡面選至少1個選手

5. 選手的ball-handling、shooting、rebounding的平均等級都要超過2(表示total要大於10)

6. 若3號上場，6號不能上

7. 若1號上場，4和5號要上場號

8. 不是2號就是3號一定要上場

9. 各變數都必須是binary


In [ ]:
m2.addConstr((x[0]+x[1]+x[2]+x[3]+x[4]+x[5]+x[6]) == 5)
m2.addConstr((x[0]+x[2]+x[4]+x[6]) >=4)# play guard
m2.addConstr((x[2]+x[3]+x[4]+x[5]+x[6]) >=2)
m2.addConstr((x[1]+x[3]+x[5]) >=1)
m2.addConstr((3*x[0])+(2*x[1])+(2*x[2])+(1*x[3])+(3*x[4])+(3*x[5])+(3*x[6])>=10)
m2.addConstr((3*x[0])+(1*x[1])+(3*x[2])+(3*x[3])+(3*x[4])+(1*x[5])+(2*x[6])>=10)
m2.addConstr((1*x[0])+(3*x[1])+(2*x[2])+(3*x[3])+(3*x[4])+(2*x[5])+(2*x[6])>=10)
m2.addConstr((x[2]+x[5] <= 1))
m2.addConstr((x[0]<=x[3]))
m2.addConstr((x[0]<=x[4]))
m2.addConstr((x[1]+x[2])>=1)
for i in range(7):
  m2.addConstr(x[i] <=1)
  m2.addConstr(x[i] >=0)

### 目標是根據上述的條件來選出防守等級最高的隊伍

In [ ]:
obj_2 = 3*x[0] + 2*x[1] + 2*x[2] + 1*x[3] + 3*x[4] + 3*x[5] +1*x[6]
m2.setObjective(obj_2 ,grb.GRB.MAXIMIZE)
m2.update()
m2.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 25 rows, 7 columns and 62 nonzeros
Model fingerprint: 0xba7dda99
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 25 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [ ]:
print(m2.Status == grb.GRB.OPTIMAL)
print(m2.display())

True
Maximize
  3.0 x[0] + 2.0 x[1] + 2.0 x[2] + x[3] + 3.0 x[4] + 3.0 x[5] + x[6]
Subject To
  R0: x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] = 5
  R1: x[0] + x[2] + x[4] + x[6] >= 4
  R2: x[2] + x[3] + x[4] + x[5] + x[6] >= 2
  R3: x[1] + x[3] + x[5] >= 1
  R4: 3.0 x[0] + 2.0 x[1] + 2.0 x[2] + x[3] + 3.0 x[4] + 3.0 x[5] + 3.0 x[6] >= 10
  R5: 3.0 x[0] + x[1] + 3.0 x[2] + 3.0 x[3] + 3.0 x[4] + x[5] + 2.0 x[6] >= 10
  R6: x[0] + 3.0 x[1] + 2.0 x[2] + 3.0 x[3] + 3.0 x[4] + 2.0 x[5] + 2.0 x[6] >= 10
  R7: x[2] + x[5] <= 1
  R8: x[0] + -1.0 x[3] <= 0
  R9: x[0] + -1.0 x[4] <= 0
  R10: x[1] + x[2] >= 1
  R11: x[0] <= 1
  R12: x[0] >= 0
  R13: x[1] <= 1
  R14: x[1] >= 0
  R15: x[2] <= 1
  R16: x[2] >= 0
  R17: x[3] <= 1
  R18: x[3] >= 0
  R19: x[4] <= 1
  R20: x[4] >= 0
  R21: x[5] <= 1
  R22: x[5] >= 0
  R23: x[6] <= 1
  R24: x[6] >= 0
General Integers
  ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]', 'x[6]']
None


### Ans:

要挑player 1 3 4 5 7

In [ ]:
for v in m2.getVars():
  print(v.VarName, v.X)

x[0] 1.0
x[1] 0.0
x[2] 1.0
x[3] 1.0
x[4] 1.0
x[5] 0.0
x[6] 1.0
